In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

**Task 1** get the data into pandas

In [2]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_data=requests.get(url).text

In [3]:
soup = BeautifulSoup(html_data, 'html5lib')

In [4]:
tables=soup.find_all("table")
len(tables)

3

In [5]:
table=tables[0]

canada_data=pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])

for row in table.find("tbody").find_all("tr"):
    cols=row.find_all("td")
    
    if len(cols)!=3:
        continue
    if "Not assigned" in cols[1].text or "Not assigned" in cols[2].text:
        continue
    canada_data=canada_data.append({"PostalCode":cols[0].text,"Borough":cols[1].text,"Neighborhood":cols[2].text}, ignore_index=True)
    
canada_data=canada_data.applymap(lambda s: s[:-1])

combine rows by PostalCode and Borough

In [6]:
canada_data=canada_data.groupby(["PostalCode","Borough"])["Neighborhood"].apply(' '.join).reset_index()

In [7]:
canada_data.shape

(103, 3)

**Task 2** coordinates 

define a function to get coordinate by postal code
failed to load cooridinates by api, 

failed to get coordiates by api, will use csf file to get coordinates

In [13]:
coordiates=pd.read_csv("https://cocl.us/Geospatial_data")
coordiates=coordiates.rename(columns={"Postal Code":"PostalCode"})
coordiates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


merge 2 dataframes 

In [14]:
canada_data=pd.merge(canada_data,coordiates)
canada_data.head()

(103, 5)

In [15]:
canada_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


**Task 3** Explore data in Toronto

In [17]:
toronto_data=canada_data[canada_data["Borough"].str.contains("Toronto")]

In [18]:
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
